In [ ]:
import pandas as pd
import statsmodels.api as sm
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.api import qqplot

def linear_regression_model(df, dependent_var, independent_vars, n_features_to_select=_):
    """
    Function performs multiple linear regression on specified variables. Displays OLS summary and calculates RSE
    and creates Q-Q plot. Feature selection/model reduction is built in with sklearn RFE. The function automatically
    takes the features selected by RFE and returns Summary, RSE, and Q-Q plot of reduced model. 

    Parameters:
    - df: pandas.DataFrame to use.
    - dependent_var: Target/Dependent variable.
    - independent_vars: Independent/Explanitory variables
    - n_features_to_select: Number of features to include/select for reduced model

    Returns:
    - OLS summary result of initial and reduced models, RSE values, Q-Q plots and variables selected by RFE function.
    """
    # Independent/explanatory variables
    X = df[independent_vars]

    # Dependent/target variable
    y = df[dependent_var]

    # Add constant/y-intercept to the model
    X = sm.add_constant(X)

    # Fit the initial model and assign to variable
    initial_model = sm.OLS(y, X).fit()

    # Display stats generated from the initial model
    print("Initial Model Summary:")
    print(initial_model.summary())

    # Calculate and print the initial model's residual standard error
    rse_initial_model = np.sqrt(initial_model.mse_resid)
    print()
    print('The initial model\'s Residual Standard Error = ', rse_initial_model)
    print()

    # Reduce the model via recursive feature elimination using sklearn.feature_selection
    linear_model = LinearRegression()
    rfe = RFE(estimator=linear_model, n_features_to_select=n_features_to_select, importance_getter='auto')
    fit = rfe.fit(X, y)

    # Print the most important features determined by sklearn RFE function
    selected_features = list(X.columns[fit.support_])
    print("Selected Features for Reduced Model: %s" % selected_features)
    print()

    # Build the reduced model using the selected features
    X_reduced = df[selected_features]
    X_reduced = sm.add_constant(X_reduced)
    reduced_model = sm.OLS(y, X_reduced).fit()

    # Display stats generated from the reduced model
    print("Reduced Model Summary:")
    print(reduced_model.summary())

    # Calculate and print the reduced model's residual standard error
    rse_reduced_model = np.sqrt(reduced_model.mse_resid)
    print()
    print('The reduced model\'s Residual Standard Error = ', rse_reduced_model)
    print()

    # Create Q-Q plot to look at distribution of residuals for the initial model
    qqplot(data=initial_model.resid, fit=True, line='45')
    plt.title("Q-Q Plot Of Initial Model")
    plt.show()

    # Create Q-Q plot to look at distribution of residuals for the reduced model
    qqplot(data=reduced_model.resid, fit=True, line='45')
    plt.title("Q-Q Plot Of Reduced Model")
    plt.show()

    return {
        "initial_model": initial_model,
        "reduced_model": reduced_model,
        "selected_features": selected_features,
        "rse_initial_model": rse_initial_model,
        "rse_reduced_model": rse_reduced_model
    }

dependent_var = 'A'
independent_vars = ['B','C','D','E']
n_features_to_select = 2  # number of features to include in reduced model

results = linear_regression_model(df, dependent_var, independent_vars, n_features_to_select=n_features_to_select)

# Access the results
initial_model = results['initial_model']
reduced_model = results['reduced_model']
selected_features = results['selected_features']
rse_initial_model = results['rse_initial_model']
rse_reduced_model = results['rse_reduced_model']
